In [2]:
import torch
import numpy as np

bikes_numpy = np.loadtxt("./hour-fixed.csv",dtype = np.float32,delimiter=",",skiprows = 1,converters = {1:lambda x:float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)

print(bikes.shape)
print(bikes)
print(bikes.stride())

torch.Size([17520, 17])
tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])
(17, 1)


In [3]:
## view 返回一个新的张量 该张量可以更改维数信息  不用更改存储 返回新的张量
daily_bikes = bikes.view(-1,24,bikes.shape[1])
print(daily_bikes.shape)
print(daily_bikes.stride())

torch.Size([730, 24, 17])
(408, 17, 1)


In [4]:
## 最右边的维度是列数   中间的维度 将时间分为24小时的块 也就是每天有C个通道的N个L小时的序列 N C L
daily_bikes = daily_bikes.transpose(1,2)
print(daily_bikes.shape)
print(daily_bikes.stride())

torch.Size([730, 17, 24])
(408, 1, 17)


In [6]:
## 初始化一个0矩阵，其行数等于一天中的小时数  列数等于天气等级的数

first_day = bikes[:24].long()  ## 取出前24小时的数据 转换为long
weather_onehot = torch.zeros(first_day.shape[0],4)  ## 24 x 4  天气等级 一开始全部为0
print(first_day[:,9])  ## 打印所有行的第十列 天气数据
print(weather_onehot)


tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [11]:
## 转换为独热编码
index_ = first_day[:,9].unsqueeze(1) - 1  ## 因为是索引 需要减一 
print(index_)
print(index_.shape)
weather_onehot.scatter_(dim = 1,index = first_day[:,9].unsqueeze(1) - 1,value = 1.0)
print(weather_onehot)
print(weather_onehot.shape)

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [1],
        [1],
        [1],
        [1]])
torch.Size([24, 1])
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])
torch.Size([24, 4])


In [13]:
print(torch.cat((bikes[:24],weather_onehot),1))

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 6.0000e+00, 0.0000e+00, 1.0000e+00, 2.4000e-01, 2.8790e-01,
         8.1000e-01, 0.0000e+00, 3.0000e+00, 1.3000e+01, 1.6000e+01, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
         0.0000e+00, 6.0000e+00, 0.0000e+00, 1.0000e+00, 2.2000e-01, 2.7270e-01,
         8.0000e-01, 0.0000e+00, 8.0000e+00, 3.2000e+01, 4.0000e+01, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 2.0000e+00,
         0.0000e+00, 6.0000e+00, 0.0000e+00, 1.0000e+00, 2.2000e-01, 2.7270e-01,
         8.0000e-01, 0.0000e+00, 5.0000e+00, 2.7000e+01, 3.2000e+01, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 3.0000e+00,
         0.0000e+00, 6.0000e+00, 0.0000e+00, 1.0000e

In [15]:
# x1
x1 = torch.tensor([[11,21,31],[21,31,41]],dtype=torch.int)
x1.shape # torch.Size([2, 3])
# x2
x2 = torch.tensor([[12,22,32],[22,32,42]],dtype=torch.int)
x2.shape  # torch.Size([2, 3])



torch.cat((x1,x2), dim=1)




tensor([[11, 21, 31, 12, 22, 32],
        [21, 31, 41, 22, 32, 42]], dtype=torch.int32)

In [21]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
daily_bikes.shape[2]) 
daily_weather_onehot.shape
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1) - 1,
                                                            1.0) 
daily_weather_onehot.shape

print(daily_weather_onehot.shape)


torch.Size([730, 4, 24])


In [20]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
print(daily_bikes.shape)

torch.Size([730, 25, 24])
